# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


# Reading Datasets
## Using file1 as the main dataset

In [2]:
file1 = 'Diabetes_Data_Sub_Strict_Final.txt'
file2 = 'Diabetes_Data_Sub_Strict_Main_String.txt'
file3 = 'Diabetes_Data_Sub_Strict_Main.txt'

# read in the file
df1 = pd.read_csv(file1, sep='\t')
df2 = pd.read_csv(file2, sep='\t')
df3 = pd.read_csv(file3, sep='\t')

In [3]:
len(df1.columns)

52

In [4]:
# Display basic information and the first few rows of the dataset
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212022 entries, 0 to 212021
Data columns (total 52 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           212022 non-null  int64  
 1   SEXVAR       212022 non-null  int64  
 2   GENHLTH      212022 non-null  int64  
 3   PHYSHLTH     212022 non-null  int64  
 4   MENTHLTH     212022 non-null  int64  
 5   PRIMINSR     212022 non-null  int64  
 6   CHECKUP1     212022 non-null  int64  
 7   EXERANY2     212022 non-null  int64  
 8   BPHIGH6      212022 non-null  int64  
 9   TOLDHI3      212022 non-null  int64  
 10  CVDINFR4     212022 non-null  int64  
 11  CVDCRHD4     212022 non-null  int64  
 12  CVDSTRK3     212022 non-null  int64  
 13  ASTHMA3      212022 non-null  int64  
 14  CHCCOPD3     212022 non-null  int64  
 15  ADDEPEV3     212022 non-null  int64  
 16  CHCKDNY2     212022 non-null  int64  
 17  DIABETE4     212022 non-null  int64  
 18  HAVARTH5     212022 non-

In [5]:
# Summary statistics
print("Summary Statistics:\n", df1.describe())

Summary Statistics:
                   ID         SEXVAR        GENHLTH       PHYSHLTH  \
count  212022.000000  212022.000000  212022.000000  212022.000000   
mean   219822.896176       1.525964       2.478281      62.473338   
std    128467.836386       0.499327       1.027709      36.727832   
min         1.000000       1.000000       1.000000       1.000000   
25%    107627.250000       1.000000       2.000000      20.000000   
50%    219215.500000       2.000000       2.000000      88.000000   
75%    332475.500000       2.000000       3.000000      88.000000   
max    438693.000000       2.000000       5.000000      88.000000   

            MENTHLTH       PRIMINSR       CHECKUP1       EXERANY2  \
count  212022.000000  212022.000000  212022.000000  212022.000000   
mean       59.631666       5.766170       1.303634       1.222901   
std        37.855572      16.129301       0.748074       0.416194   
min         1.000000       1.000000       1.000000       1.000000   
25%        1

## Target Variable

In [6]:
df1['DIABETE4'].value_counts()

3    176116
1     30863
4      5043
Name: DIABETE4, dtype: int64

# Scale down 52 features to 40 features, so keep the most important features

In [7]:
# # Identify highly correlated features (correlation greater than 0.9)
# high_corr_pairs = correlation_matrix.abs().stack().reset_index()
# high_corr_pairs.columns = ['Feature1', 'Feature2', 'Correlation']
# high_corr_pairs = high_corr_pairs[(high_corr_pairs['Feature1'] != high_corr_pairs['Feature2']) & (high_corr_pairs['Correlation'] > 0.9)]
# # Sort the pairs by correlation to see the most correlated pairs
# high_corr_pairs.sort_values(by='Correlation', ascending=False, inplace=True)
# print("Highly correlated pairs:")
# print(high_corr_pairs)

In [8]:
# Define the target variable
target = 'DIABETE4'

# Calculate the correlation matrix of the dataframe
correlation_matrix = df1.corr()

# Isolate the correlations of all features with the target variable
target_correlations = correlation_matrix[target].drop(target)  # Drop self-correlation

# Define a threshold for 'irrelevance'
threshold = 0.01

# Identify features where the absolute value of the correlation coefficient is below the threshold
irrelevant_features = target_correlations[abs(target_correlations) < threshold].index.tolist()

# Output the list of irrelevant features
print("Irrelevant features to drop based on threshold of", threshold, ":\n", irrelevant_features)

# Drop these features from the dataframe
df1_final = df1.drop(columns=irrelevant_features)

Irrelevant features to drop based on threshold of 0.01 :
 ['MARITAL', 'RENTHOM1', 'HTM4', 'X_RACE', 'X_FRUTSU1', 'X_VEGSU1', 'X_FRUTSU1D', 'X_VEGSU1D']


In [9]:
# Show the first few rows of the updated dataframe
df1_final.head()

,ID,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,PRIMINSR,CHECKUP1,EXERANY2,BPHIGH6,TOLDHI3,...,CHOLMED3,CHCOCNCR,PHYSHLTH14D,MENTHLTH14D,ALCOCONS,ALCOFREQ,VACCSTAT,CHOLSTAT,X_FRUTSU1DF,X_VEGSU1DF
0,1,2,5,20,10,3,2,2,3,1,...,1,2,3,2,0,1,1,1,2,2
1,3,2,2,88,88,2,1,2,1,2,...,2,2,1,1,0,1,4,4,2,1
2,4,2,2,88,10,2,1,1,1,1,...,2,2,1,2,4,2,3,2,2,1
3,6,1,3,88,88,3,1,2,3,2,...,2,2,1,1,0,1,1,4,1,1
4,10,2,3,25,5,3,1,1,1,2,...,1,2,3,2,3,2,2,3,2,1


# Calculating the correlation between the features 

## calculate the association between different variables and our Output variable by calculating Odds Ratio and Logs Odd Ratio

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Load data
X = df1_final.drop('DIABETE4', axis=1)  # Adjust the 'target_column' to your dataset's target feature
y = df1_final['DIABETE4']

# Split data into training and testing sets with a 70/30 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Applying RFE
model = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence if needed
rfe = RFE(model, n_features_to_select=40)
rfe.fit(X_train_scaled, y_train)

# Selected features
selected_features = X.columns[rfe.support_]
print("Selected features via RFE:", selected_features)

# Train weighted logistic regression on selected features
weighted_model = LogisticRegression(class_weight='balanced', max_iter=1000)
weighted_model.fit(X_train_scaled[:, rfe.support_], y_train)  # Training only on selected features


Selected features via RFE: Index(['ID', 'SEXVAR', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'PRIMINSR',
       'CHECKUP1', 'EXERANY2', 'BPHIGH6', 'TOLDHI3', 'CVDINFR4', 'CVDCRHD4',
       'CVDSTRK3', 'ASTHMA3', 'CHCCOPD3', 'ADDEPEV3', 'CHCKDNY2', 'HAVARTH5',
       'EMPLOY1', 'INCOME3', 'WEIGHT2', 'BLIND', 'DECIDE', 'PNEUVAC4',
       'DROCDY3_', 'X_AGEG5YR', 'X_EDUCAG', 'X_TOTINDA', 'X_ASTHMS1',
       'X_BMI5CAT', 'CHOLCHK3', 'CHOLMED3', 'CHCOCNCR', 'PHYSHLTH14D',
       'ALCOCONS', 'ALCOFREQ', 'VACCSTAT', 'CHOLSTAT', 'X_FRUTSU1DF',
       'X_VEGSU1DF'],
      dtype='object')


LogisticRegression(class_weight='balanced', max_iter=1000)

In [11]:
# Calculate Odds Ratios and Log Odds Ratios
coefficients = pd.DataFrame({
    'Feature': selected_features,
    'Coefficient': weighted_model.coef_[0],
    'Odds Ratio': np.exp(weighted_model.coef_[0]),
    'Log Odds Ratio': np.log(np.exp(weighted_model.coef_[0]))
})

# Sort features by Odds Ratio
coefficients.sort_values(by='Odds Ratio', ascending=False, inplace=True)

print(coefficients)


        Feature  Coefficient  Odds Ratio  Log Odds Ratio
2       GENHLTH     0.261281    1.298592        0.261281
20      WEIGHT2     0.106317    1.112174        0.106317
29    X_BMI5CAT     0.078676    1.081854        0.078676
9       TOLDHI3     0.068844    1.071269        0.068844
25    X_AGEG5YR     0.056287    1.057901        0.056287
17     HAVARTH5     0.052182    1.053567        0.052182
14     CHCCOPD3     0.042391    1.043303        0.042391
4      MENTHLTH     0.033747    1.034323        0.033747
27    X_TOTINDA     0.026831    1.027195        0.026831
7      EXERANY2     0.026831    1.027195        0.026831
22       DECIDE     0.025852    1.026189        0.025852
39   X_VEGSU1DF     0.022623    1.022881        0.022623
5      PRIMINSR     0.015038    1.015152        0.015038
32     CHCOCNCR     0.014995    1.015108        0.014995
0            ID     0.013399    1.013489        0.013399
18      EMPLOY1    -0.001234    0.998767       -0.001234
26     X_EDUCAG    -0.002657   

In [12]:
# keep only the selected features and the target variable 
df2 = df1_final[selected_features.union(['DIABETE4'])]

In [13]:
df2.head()


,ADDEPEV3,ALCOCONS,ALCOFREQ,ASTHMA3,BLIND,BPHIGH6,CHCCOPD3,CHCKDNY2,CHCOCNCR,CHECKUP1,...,TOLDHI3,VACCSTAT,WEIGHT2,X_AGEG5YR,X_ASTHMS1,X_BMI5CAT,X_EDUCAG,X_FRUTSU1DF,X_TOTINDA,X_VEGSU1DF
0,2,0,1,1,2,3,1,2,2,2,...,1,1,72,11,1,1,2,2,2,2
1,2,0,1,2,2,1,2,2,2,1,...,2,4,170,11,3,3,2,2,2,1
2,2,4,2,2,2,1,2,2,2,1,...,1,3,195,9,3,4,2,2,1,1
3,2,0,1,2,2,3,1,2,2,1,...,2,1,195,13,3,2,3,1,2,1
4,2,3,2,2,2,1,2,2,2,1,...,2,2,240,10,3,4,2,2,1,1


# Training Models on new Data

In [14]:
import numpy as np


class MyLinearRegression:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        # Adding a column of ones to input features to include the intercept in the model
        X_b = np.hstack([np.ones((X.shape[0], 1)), X])
        # Using the Normal Equation to compute coefficients
        self.coefficients = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

    def predict(self, X):
        # Adding a column of ones to input features
        X_b = np.hstack([np.ones((X.shape[0], 1)), X])
        # Predicting values using the linear model
        return X_b.dot(self.coefficients)

    def get_coefficients(self):
        # Returns the coefficients of the model; first one is the intercept
        return self.coefficients

In [15]:
import numpy as np
from collections import Counter


class MyKNN:
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        # Compute the Euclidean distance from x to all samples in the training set
        distances = [np.sqrt(np.sum((x_train - x) ** 2))
                     for x_train in self.X_train]
        # Get the k nearest samples, labels
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        # Majority vote, most common class label
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

In [20]:
import numpy as np
from scipy.stats import mode
from joblib import Parallel, delayed


class MyRandomForest:
    def __init__(self, n_estimators=100, max_samples=0.8, n_jobs=-1):
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.trees = []
        self.n_jobs = n_jobs

    def fit(self, X, y):
        n_samples = int(self.max_samples * X.shape[0])
        self.trees = Parallel(n_jobs=self.n_jobs)(
            delayed(self._fit_tree)(X, y, n_samples) for _ in range(self.n_estimators)
        )

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        predictions = mode(tree_preds, axis=0)[0]  # majority vote
        return predictions.flatten()

    def _fit_tree(self, X, y, n_samples):
        idx = np.random.choice(
            range(X.shape[0]), size=n_samples, replace=True)
        X_sample, y_sample = X[idx], y[idx]
        tree = MyDecisionStump()
        tree.fit(X_sample, y_sample)
        return tree


class MyDecisionStump:
    def __init__(self):
        self.split_feature = None
        self.threshold = None
        self.prediction = None

    def fit(self, X, y):
        # Very simple decision stump that only checks one threshold per feature
        n_features = X.shape[1]
        best_error = float('inf')
        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                prediction, error = self._try_split(
                    X, y, feature_index, threshold)
                if error < best_error:
                    best_error = error
                    self.split_feature = feature_index
                    self.threshold = threshold
                    self.prediction = prediction

    def _try_split(self, X, y, feature_index, threshold):
        # Split and measure the Gini impurity or another metric
        left_mask = X[:, feature_index] <= threshold
        right_mask = ~left_mask
        left_labels, right_labels = y[left_mask], y[right_mask]
        left_mode = mode(left_labels)[0] if len(left_labels) > 0 else None
        right_mode = mode(right_labels)[0] if len(right_labels) > 0 else None
        left_error = np.sum(left_labels != left_mode)
        right_error = np.sum(right_labels != right_mode)
        error = left_error + right_error
        return (left_mode if len(left_labels) > len(right_labels) else right_mode), error

    def predict(self, X):
        # Predict based on the feature and threshold learned
        if X[:, self.split_feature] <= self.threshold:
            return np.full(X.shape[0], self.prediction)
        else:
            # Simplistic: assumes binary target
            return np.full(X.shape[0], 1 - self.prediction)

In [ ]:

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.verbose = verbose
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        self.weights = np.zeros(X.shape[1])
        self.bias = 0

        for i in range(self.num_iterations):
            # Compute predictions and errors
            y_pred = self._sigmoid(np.dot(X, self.weights) + self.bias)
            errors = y_pred - y

            # Update weights and bias
            self.weights -= self.learning_rate * np.dot(X.T, errors)
            self.bias -= self.learning_rate * np.sum(errors)

            # Print progress
            if self.verbose and i % 100 == 0:
                print(f'Iteration {i}, Loss: {self._compute_loss(y, y_pred)}')

    def predict(self, X):
        y_pred = self._sigmoid(np.dot(X, self.weights) + self.bias)
        return (y_pred > 0.5).astype(int)

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _compute_loss(self, y_true, y_pred):
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [21]:
# %%
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import label_binarize

# Load the dataset
df2 = pd.read_csv('Sample.csv')

# Prepare the data
# Adjust 'DIABETE4' if your target variable is named differently
X = df2.drop('DIABETE4', axis=1)
y = df2['DIABETE4']

# Binarize the output
y = label_binarize(y, classes=[1, 3, 4])
n_classes = y.shape[1]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
models = {
    "My Linear Regression": MyLinearRegression(),
    "My K Nearest Neighbors": MyKNN(k=5),
    "My Random Forest": MyRandomForest(n_estimators=100, max_samples=0.8),
    "My Logistic Regression": LogisticRegression(learning_rate=0.01, num_iterations=1000, verbose=False)
}

# Train, predict, evaluate, and save each model
for name, model in models.items():
    print(f"Training and evaluating {name}...")
    # Fitting model
    if name == "My Linear Regression":
        for i in range(n_classes):
            model.fit(X_train_scaled, y_train[:, i])
            y_pred = model.predict(X_test_scaled)
            # Convert continuous predictions to binary
            y_pred_binary = label_binarize(y_pred >= 0.5, classes=[0, 1])
            # Print and evaluate predictions
            accuracy = accuracy_score(y_test[:, i], y_pred_binary)
            print(f"Class {i} - Accuracy: {accuracy}")
    else:
        model.fit(X_train_scaled, y_train.argmax(axis=1))
        y_pred = model.predict(X_test_scaled)
        y_pred_binary = label_binarize(y_pred, classes=[0, 1, 2])
        # Metrics and results
        accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
        precision = precision_score(y_test.argmax(
            axis=1), y_pred, average='macro', zero_division=0)
        recall = recall_score(y_test.argmax(axis=1), y_pred, average='macro')
        f1 = f1_score(y_test.argmax(axis=1), y_pred, average='macro')
        cm = confusion_matrix(y_test.argmax(axis=1), y_pred)

        print(f"Results for {name}:")
        print("Accuracy: {:.4f}".format(accuracy))
        print("Precision: {:.4f}".format(precision))
        print("Recall: {:.4f}".format(recall))
        print("F1 Score: {:.4f}".format(f1))
        print("Confusion Matrix:\n", cm)

    print("\n" + "-"*60 + "\n")

    # Save the model using pickle
    with open(f'{name.replace(" ", "_")}_model.pkl', 'wb') as file:
        pickle.dump(model, file)

Training and evaluating My Linear Regression...
Class 0 - Accuracy: 0.608376436555725
Class 1 - Accuracy: 0.6948134639269263
Class 2 - Accuracy: 0.6236892165956577

------------------------------------------------------------

Training and evaluating My K Nearest Neighbors...
